In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv('../datasets/train.csv')
test = pd.read_csv('../datasets/test.csv')
print(train.shape)
train.head()

(42000, 785)


,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
x_train = (train.ix[:,1:].values).astype('float32') # all pixel values
y_train = train.ix[:,0].values.astype('int32') # only labels i.e targets digits
x_test = test.values.astype('float32')

In [4]:
x_train = x_train/255.0
x_test = x_test/255.0

In [6]:
X_train = x_train.reshape(x_train.shape[0], 28, 28,1)
X_test = x_test.reshape(x_test.shape[0], 28, 28,1)

In [13]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import RMSprop
from keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split

In [8]:
batch_size = 64
num_classes = 10
epochs = 15
input_shape = (28, 28, 1)

In [9]:
y_train = keras.utils.to_categorical(y_train, num_classes)

In [10]:
X_train, X_val, Y_train, Y_val = train_test_split(X_train, y_train, test_size = 0.1, random_state=42)

In [11]:
model = Sequential()
model.add(Conv2D(32,(3,3), activation='relu',padding='same',input_shape=input_shape))
model.add(Conv2D(32,(3,3),activation='relu',padding='same'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(.2))

model.add(Conv2D(64,(3,3),activation='relu',padding='same'))
model.add(Conv2D(64,(3,3),activation='relu',padding='same'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(.2))

model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(.2))
model.add(Dense(num_classes,activation='softmax'))

In [14]:
model.compile(optimizer=RMSprop(lr=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy'])

In [15]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.0, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images

In [17]:
datagen.fit(X_train)

In [18]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [19]:
h = model.fit_generator(datagen.flow(X_train,Y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (X_val,Y_val),
                              verbose = 2, steps_per_epoch=X_train.shape[0] // batch_size
                              , callbacks=[learning_rate_reduction])

Epoch 1/15
 - 337s - loss: 0.2605 - acc: 0.9187 - val_loss: 0.0681 - val_acc: 0.9786
Epoch 2/15
 - 317s - loss: 0.0944 - acc: 0.9701 - val_loss: 0.0454 - val_acc: 0.9855
Epoch 3/15
 - 322s - loss: 0.0772 - acc: 0.9767 - val_loss: 0.0420 - val_acc: 0.9860
Epoch 4/15
 - 350s - loss: 0.0665 - acc: 0.9802 - val_loss: 0.0880 - val_acc: 0.9745
Epoch 5/15
 - 316s - loss: 0.0581 - acc: 0.9819 - val_loss: 0.0553 - val_acc: 0.9829
Epoch 6/15
 - 316s - loss: 0.0544 - acc: 0.9826 - val_loss: 0.0353 - val_acc: 0.9910
Epoch 7/15
 - 317s - loss: 0.0513 - acc: 0.9847 - val_loss: 0.0279 - val_acc: 0.9926
Epoch 8/15
 - 315s - loss: 0.0474 - acc: 0.9854 - val_loss: 0.0292 - val_acc: 0.9912
Epoch 9/15
 - 314s - loss: 0.0428 - acc: 0.9870 - val_loss: 0.0229 - val_acc: 0.9940
Epoch 10/15
 - 311s - loss: 0.0440 - acc: 0.9862 - val_loss: 0.0222 - val_acc: 0.9926
Epoch 11/15
 - 306s - loss: 0.0427 - acc: 0.9869 - val_loss: 0.0277 - val_acc: 0.9919
Epoch 12/15
 - 309s - loss: 0.0410 - acc: 0.9868 - val_loss: 0.

In [23]:
model.save('../model/keras_model.h5')

In [20]:
score = model.evaluate(X_val, Y_val, verbose=1)
print('valid loss:', score[0])
print('valid accuracy:', score[1])

4200/4200 [==============================] - 11s 3ms/step

In [21]:
y_pred = model.predict(X_test)

In [22]:
y_pred_classes = np.argmax(y_pred,axis = 1)
ids = range(1,len(y_pred_classes)+1)
submission = pd.DataFrame(np.column_stack((ids,y_pred_classes)),columns=("ImageId","Label"))
submission.to_csv("Keras_mnist.csv", index=None)